In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools.traveltime.models import rnn
from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

### Energy Use for Full KCM Network

In [ ]:
network_name = "kcm"

res_dir = Path("..","results","energy", network_name)

file = open(res_dir / "trajectories.pkl", "rb")
trajectories = pickle.load(file)
file.close()

file = open(res_dir / "cycles.pkl", "rb")
cycles = pickle.load(file)
file.close()

In [ ]:
efficiencies = np.array([1/cycle.electric_kwh_per_mi for cycle in cycles])
fig, ax = plt.subplots()
sns.histplot(efficiencies[efficiencies<5], kde=True, ax=ax)
ax.set_title(f"BEB Trip Efficiency Distribution\n{len(efficiencies)} Routes")
ax.set_xlabel("Efficiency (mi/kWh)")
plt.show()

In [ ]:
# static_dir = Path("..","data","kcm_static","2023_09_27")
# target_day = "2024_01_03"
# t_day_of_week = datetime.strptime(target_day, "%Y_%m_%d").weekday()
# static_feed = gk.read_feed(static_dir, dist_units="km").restrict_to_dates([str.replace(target_day,'_','')])
# active_service_ids = static_feed.calendar[static_feed.calendar.iloc[:,t_day_of_week+1]==1]['service_id'].to_numpy()
# trips = static_feed.get_trips()
# trips = trips[trips['service_id'].isin(active_service_ids)]
# trip_start_end = standardfeeds.get_trip_start_and_end_times(static_feed)
# trips = pd.merge(trips, trip_start_end, on='trip_id')
# trips

In [ ]:
# Get energy consumption for all trips in a block
total_energy_kwh = [np.sum(c.ess_kw_out_ach) / 3600 for c in cycles]
trip_id = [t.traj_attr['trip_id'] for t in trajectories]
block_id = [t.traj_attr['block_id'] for t in trajectories]
energy_res = pd.DataFrame({'trip_id': trip_id, 'block_id': block_id, 'total_energy_kwh': total_energy_kwh})
energy_res = energy_res.sort_values(['block_id','trip_id'])[['block_id','trip_id','total_energy_kwh']].copy()
energy_res = energy_res.groupby('block_id').agg({'total_energy_kwh': 'sum', 'trip_id': 'count'}).reset_index().sort_values('total_energy_kwh')
energy_res

### Draw Energy Predictions for All Networks

In [ ]:
# Does a higher GTFS-RT sampling frequency improve energy estimates of the model?
# Which blocks are BEBs viable on, and what is the cost in infrastructure of electrifying them?

# How accurately can bus energy consumption be predicted using a combination of route
# geometry, elevation and models trained on standardized open bus data to inform system
# design for transit agencies transitioning to BEBs?
# 2. Given demand estimates from different sources (AVL, GTFS-RT, onboard logger) com-
# bined with a power consumption modeling framework:
# (a) Which blocks are BEBs viable on?
# (b) How do design parameters such as battery and charger sizing affect the decision
# of which blocks to electrify?
# (c) How does the availability of data sources (e.g., static only, static and realtime
# sample) impact energy predictions?

# Calculate operating parameters from GTFS + TT model
# Given GTFS energy, operating profile, design strategy, cost assumptions: Compare energy needs/electrification barriers across wide array of agencies
# Calculate drive cycle/energy use for full GTFS feed
# Growth in GTFS-RT, GTFS-Flex, API sampling resolutions, Simulate/propose change in GTFS-RT sampling frequency
# Where is GPS precision high/low in network? Energy implications?
# Energy models as constraint in blocking software
# BEB abatement curve?

In [ ]:
cleaned_sources = pd.read_csv(Path('..', 'data', 'cleaned_sources.csv'))
all_network_trajs = []
all_network_cycles = []
for i, row in cleaned_sources.iterrows():
    try:
        network_name = row['uuid']
        res_dir = Path("..","results","energy", network_name)

        file = open(res_dir / "trajectories.pkl", "rb")
        trajectories = pickle.load(file)
        file.close()
        all_network_trajs.append(trajectories)

        file = open(res_dir / "cycles.pkl", "rb")
        cycles = pickle.load(file)
        file.close()
        all_network_cycles.append(cycles)
    except FileNotFoundError:
        print(f"Network {network_name} not found")

In [ ]:
df = []
for i, network_cycles in enumerate(all_network_cycles):
    for cycle in network_cycles:
        df.append({"efficiency": 1/cycle.electric_kwh_per_mi, "network": cleaned_sources.iloc[i]['provider'], "num_shapes": len(network_cycles)})
df = pd.DataFrame(df)
df_sorted = df.groupby("network").median().sort_values("efficiency")
df

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(18,5))
sns.boxplot(x="network", y="efficiency", data=df[df['efficiency']<1], order=df_sorted.index)
plt.xticks(rotation=90)
plt.xlabel("")
plt.ylabel("Efficiency (mi/kWh)")
plt.show()